In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [2]:
data ='https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv'

In [3]:
!wget $data -O homework.csv 

--2025-10-15 17:52:28--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘homework.csv’

homework.csv        100%[===================>]  78.98K  --.-KB/s    in 0.07s   

2025-10-15 17:52:28 (1.05 MB/s) - ‘homework.csv’ saved [80876/80876]



In [74]:
df = pd.read_csv('homework.csv')
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


## Data Preparation

In [75]:
df.isnull().sum()

lead_source                 128
industry                    134
number_of_courses_viewed      0
annual_income               181
employment_status           100
location                     63
interaction_count             0
lead_score                    0
converted                     0
dtype: int64

In [76]:
categorical = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical:
    df[c] = df[c].fillna('NA')

In [77]:
df.columns

Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score',
       'converted'],
      dtype='object')

In [78]:
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [79]:
numerical = ['number_of_courses_viewed','annual_income','lead_score','interaction_count']

for c in numerical:
    df[c] = df[c].fillna(0.0)

In [80]:
df.isnull().sum()

lead_source                 0
industry                    0
number_of_courses_viewed    0
annual_income               0
employment_status           0
location                    0
interaction_count           0
lead_score                  0
converted                   0
dtype: int64

#### Question 1: Mode of Industry column

In [81]:
df['industry'].mode()

0    retail
Name: industry, dtype: object

## Correlation Matrix

In [82]:
from sklearn.metrics import mutual_info_score

#### Question 2: Correlation for numerical

In [83]:
for first in ['interaction_count','number_of_courses_viewed','annual_income']:
    for second in ['lead_score','interaction_count']:
        mi = mutual_info_score(df[first],df[second])
        print(first,second, mi)

interaction_count lead_score 0.28943458886967544
interaction_count interaction_count 1.9046073270060195
number_of_courses_viewed lead_score 0.22982932812563833
number_of_courses_viewed interaction_count 0.025339813694093053
annual_income lead_score 4.045487068460246
annual_income interaction_count 1.6555734349605151


/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/cluster/_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received multiclass values for label, and continuous values for target
  warnings.warn(msg, UserWarning)
/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/cluster/_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received multiclass values for label, and continuous values for target
  warnings.warn(msg, UserWarning)
/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/cluster/_supervised.py:49: UserWarning: The number of unique classes is greater than 50% of the number of samples.
  type_label = type_of_target(labels_true)
/home/codespace/.local/lib/python3.12/site-packages/sklearn/metrics/cluster/_supervised.py:58: UserWarning: Clustering metrics expects discrete values but received multiclass values for label, and continuous values for target
  warnings.warn(msg, UserWar

## Validation Framework

In [84]:
from sklearn.model_selection import train_test_split

In [85]:
df_full_train, df_test = train_test_split(df,test_size = 0.2, random_state = 42)

In [86]:
df_train, df_val = train_test_split(df_full_train, test_size = 0.25, random_state = 42)

In [87]:
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [88]:
df_train = df_train.reset_index(drop = True)
df_val = df_val.reset_index(drop = True)
df_test = df_test.reset_index(drop = True)

In [89]:
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

del df_train['converted']
del df_val['converted']
del df_test['converted']

In [90]:
def mutual_info_converted_score(Series):
    return round(mutual_info_score(Series, y_train),2)

#### Question 3: Correlation for categorical

In [91]:
mi = df_train[categorical].apply(mutual_info_converted_score)
mi.sort_values(ascending= False)

lead_source          0.04
industry             0.01
employment_status    0.01
location             0.00
dtype: float64

## Train a model

In [92]:
from sklearn.feature_extraction import DictVectorizer

In [93]:
train_dicts = df_train[categorical+numerical].to_dict(orient = 'records')

In [94]:
dv = DictVectorizer(sparse = False)
X_train = dv.fit_transform(train_dicts)

In [95]:
val_dicts = df_val[categorical+numerical].to_dict(orient = 'records')
X_val = dv.transform(val_dicts)

In [96]:
from sklearn.linear_model import LogisticRegression

In [97]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,42
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [103]:
y_pred = model.predict_proba(X_val)[:, 1]
churn_decision = (y_pred >= 0.5)
original_accuracy = (y_val == churn_decision).mean()
round((y_val == churn_decision).mean(),2)

np.float64(0.7)

In [105]:
X_train

array([[5.8472e+04, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [7.1738e+04, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        3.0000e+00],
       [8.1973e+04, 0.0000e+00, 1.0000e+00, ..., 1.0000e+00, 0.0000e+00,
        3.0000e+00],
       ...,
       [8.9042e+04, 0.0000e+00, 1.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        3.0000e+00],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [5.0259e+04, 0.0000e+00, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        4.0000e+00]], shape=(876, 31))

In [115]:
for feature in categorical+numerical:
    df_train_small = df_train.drop(columns = feature)
    train_dicts_small = df_train_small.to_dict(orient = 'records')
    X_train_small = dv.fit_transform(train_dicts_small)

    df_val_small = df_val.drop(columns = feature)
    val_dicts_small = df_val_small.to_dict(orient = 'records')
    X_val_small = dv.transform(val_dicts_small)
    
    model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
    model.fit(X_train_small, y_train)
    y_pred = model.predict_proba(X_val_small)[:, 1]
    churn_decision = (y_pred >= 0.5)
    modified_accuracy = (y_val == churn_decision).mean()
    accuracy_change = original_accuracy - modified_accuracy
    print(feature,accuracy_change)

lead_source -0.0034129692832765013
industry 0.0
employment_status 0.0034129692832763903
location -0.010238907849829393
number_of_courses_viewed 0.14334470989761094
annual_income -0.15358361774744034
lead_score -0.0068259385665528916
interaction_count 0.14334470989761094


In [126]:
for c in [0.01, 0.1, 1, 10, 100]:
    model = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_val)[:, 1]
    churn_decision = (y_pred >= 0.5)
    accuracy = round((y_val == churn_decision).mean(),3)
    print(c,accuracy)

0.01 0.7
0.1 0.7
1 0.7
10 0.7
100 0.7
